### Graph Neural Network (GNN)

$$h_{i}^{t+1} = f\left(h_{i}^{t} W + \sum_{j \in N (i)} \frac{1}{C_{i,j}} h_{j}^{t} U \right)$$

Old representation times a weight matrix:
$$h_{i}^{t} W $$

Information from neighbors times a weight matrix:
$$h_{j}^{t} U $$

### Aggregation function:

Sum over all transformed neighbor representations

$$\sum_{j \in N (i)} \frac{1}{C_{i,j}}$$

Normalize the vectors differently for each neighbor
$$\frac{1}{C_{i,j}}$$

The sum is a $permutation-invariant$ aggregation function -> Insensitive to $order$

Each node's updated value becomes a weighting of its previous value+weightning of it's neighbors values.

-> Agg function can be mean, max, concatenation, etc.



Collapse $W_{self}$ and $W_{neigh}$ into $W$ by adding self-loops to the adjacency matrix $A$:
$$H^{(k+1)} = \sigma \left( (A+I)H^{(k)} W^{k+1} \right)$$

In [ ]:
!pip install torch
!pip install torch-geometric
!pip install torchviz

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, Linear, GATConv

import torch_geometric.transforms as T
from torch_geometric.utils import to_undirected
from torchviz import make_dot

import networkx as nx
from networkx.classes.function import density, degree
from scipy.stats import pearsonr

import csv
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 360

### Graph dataset
<div style="text-align:center; display: flex; justify-content: center;">
  <table>
    <tr>
      <th>Rosette number</th>
      <th>Nodes</th>
      <th>Edges</th>
    </tr>
    <tr>
      <td>3</td>
      <td>11157</td>
      <td>7572</td>
    </tr>
    <tr>
      <td>6</td>
      <td>9568</td>
      <td>5245</td>
    </tr>
    <tr>
      <td>7</td>
      <td>11635</td>
      <td>9257</td>
    </tr>
    <tr>
      <td>11</td>
      <td>13667</td>
      <td>13051</td>
    </tr>
    <tr>
      <td>12</td>
      <td>10617</td>
      <td>6870</td>
    </tr>
    <tr>
      <td>13</td>
      <td>13260</td>
      <td>14395</td>
    </tr>
    <tr>
      <td>14</td>
      <td>10704</td>
      <td>7635</td>
    </tr>
    <tr>
      <td>15</td>
      <td>10131</td>
      <td>8655</td>
    </tr>
    <tr>
      <td>18</td>
      <td>11117</td>
      <td>7991</td>
    </tr>
    <tr>
      <td>19</td>
      <td>10248</td>
      <td>6689</td>
    </tr>
  </table>
</div>



In [ ]:
def graph_r(r):

    nodes = []
    edges = []
    mass = []

    with open(f'/data/rosette{r}_nodes.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if (row!=0):
                values = list(row.values())
                n = []
                n.append(float(values[0]))
                n.extend(22.5-2.5*np.log10([float(n) for n in values[2:-1]]))
                n.append(float(values[-1]))
                nodes.append(n)
                mass.append(float(values[1]))

    with open(f'/data/rosette{r}_edges.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            if (row!=0):
                edges.append([float(n) for n in list(row.values())])

    return (nodes,edges,mass)

In [ ]:
rosettes = [3,6,7,11,12,13,14,15,18,19]
props = ['log flux_g','log flux_r','log flux_z','log flux_w1','log flux_w2','z']

In [ ]:
g = []
masses = []
for r in rosettes[:2]:
  nodes, edges, mass = graph_r(r)
  masses.append(mass)
  graph = nx.Graph()
  for i in range(len(nodes)):
      graph.add_node(i, attr=nodes[i][1:])

  id_to_position = {node[0]: i for i, node in enumerate(nodes)}
  mapped_edges = [[id_to_position[edge[0]], id_to_position[edge[1]], edge[2]] for edge in edges]
  for edge in mapped_edges:
      graph.add_edge(edge[0], edge[1], weight=edge[2])

  graph = graph.to_undirected()
  g.append(graph)

In [ ]:
def graph_info(graph):
  print(f'Number of nodes: {graph.number_of_nodes()}')
  print(f'Number of edges: {graph.number_of_edges()}')
  print(f'Directed: {graph.is_directed()}')
  print(f'Multigraph: {graph.is_multigraph()}')
  print(f'Average degree: {np.mean(np.array(degree(graph))[:,1]):.3f}')
  print(f'Max degree: {max(np.array(degree(graph))[:,1])}')
  print(f'Min degree: {min(np.array(degree(graph))[:,1])}')
  print(f'Density: {density(graph):.5f}')

graph_info(g[1])

Number of nodes: 9568
Number of edges: 4836
Directed: False
Multigraph: False
Average degree: 1.011
Max degree: 14
Min degree: 0
Density: 0.00011


In [ ]:
graphs = []
for gr in g:
  edge_index = torch.tensor(list(gr.edges), dtype=torch.long).t().contiguous()
  edge_attr = torch.tensor([[gr[u][v]['weight']] for u, v in gr.edges], dtype=torch.float)
  x = torch.tensor([gr.nodes[i]['attr'] for i in range(len(gr.nodes))], dtype=torch.float)
  x = F.normalize(x, dim=0)
  mass = torch.tensor([masses[g.index(gr)][i] for i in range(len(gr.nodes))], dtype=torch.float)

  num_nodes = gr.number_of_nodes()
  train_percentage = 0.8
  train_mask = torch.zeros(num_nodes, dtype=torch.bool)
  num_train_nodes = int(num_nodes*train_percentage)
  train_mask[:num_train_nodes] = True
  test_mask = ~train_mask

  graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=mass, train_mask=train_mask, test_mask=test_mask)
  graphs.append(graph)

In [ ]:
print(f'Number of training nodes: {graph.train_mask.sum()}')
print(f'Number of testing nodes: {graph.test_mask.sum()}')
print(f'Has isolated nodes: {graph.has_isolated_nodes()}')
print(f'Training node label rate: {int(graph.train_mask.sum()) / graph.num_nodes:.2f}')
#graph.validate(raise_on_error=True)

Number of training nodes: 7654
Number of testing nodes: 1914
Has isolated nodes: True
Training node label rate: 0.80


### GNN Model

In [ ]:
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_rate=0.5):
        super(GNN, self).__init__()

        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.convs = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers)])
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers)])
        self.conv_out = GCNConv(hidden_dim, output_dim)
        self.dropout = dropout_rate

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = self.bn1(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        for conv, bn in zip(self.convs, self.bn_layers):
            x = F.relu(conv(x, edge_index, edge_attr))
            x = bn(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv_out(x, edge_index, edge_attr)

        return x

In [ ]:
input_dim, hidden_dim, output_dim, num_layers = 6, 64, 1, 3
model = GNN(input_dim, hidden_dim, output_dim, num_layers, dropout_rate=0.5)

print(model)
print(f'Parameters: {sum(p.numel() for p in model.parameters())}')
tcv = make_dot(model(graphs[0]), params=dict(list(model.named_parameters()))).render("./torchviz/gnn_torchviz", format="png")

GNN(
  (conv1): GCNConv(6, 64)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convs): ModuleList(
    (0-2): 3 x GCNConv(64, 64)
  )
  (bn_layers): ModuleList(
    (0-2): 3 x BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_out): GCNConv(64, 1)
)
Parameters: 13505


Move to GPU (if available)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
graph = []
for g in graphs:
  g = g.to(device)
  graph.append(g)
device

device(type='cuda', index=0)

Optimizer: Stochastic gradient descent

Loss function: Mean squared error

In [ ]:
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
criterion = nn.MSELoss()

### Training

In [ ]:
def train(num_epochs):
      l, c = [], []

      for epoch in range(num_epochs):
          for g in graph:
              model.train()
              optimizer.zero_grad()
              out = model(g)
              train_mask, test_mask = g.train_mask, g.test_mask

              loss = criterion(out[train_mask].squeeze(), g.y[train_mask])
              loss.backward()
              optimizer.step()

              model.eval()
              with torch.no_grad():
                    pred = model(g).squeeze()
                    mse = criterion(pred[test_mask].squeeze(), g.y[test_mask])
                    pc = pearsonr(pred[test_mask].squeeze().cpu(), g.y[test_mask].cpu())
                    accuracy = torch.abs(pred - g.y).mean()
                    l.append(mse)
                    c.append(pc)

              if epoch%1==0:
                    print(f'Epoch {epoch}, Loss: {loss.item():.4f}, MSE: {mse.item():.4f}, PC: {pc.statistic:.4f}') #, Accuracy: {accuracy.item():.4f}')
      return l, c, pred

In [ ]:
losses, pcs, pred = train(100)

ValueError: ignored

In [ ]:
losses_float = [float(loss.cpu().detach().numpy()) for loss in losses]
loss_indices = [i for i,l in enumerate(losses_float)]
plt.scatter(loss_indices, losses_float, s=0.8)
plt.title('Loss function (mean squared error)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
indices = [i for i,l in enumerate(pcs)]
corr = [pc.statistic for pc in pcs]
plt.scatter(indices, corr, s=0.8)
plt.title('Pearson correlation')
plt.xlabel('Epoch')
plt.ylabel('Correlation')
plt.show()

### Testing

In [ ]:
nrows, ncols = 3, 2
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 8))
plt.tight_layout()
plt.suptitle(f'GNN model\nmse: {losses[-1]:.4f}', y=1.05)
plt.subplots_adjust(wspace=0.3, hspace=0.3)
cmap = plt.get_cmap('viridis')

pred = pred.to('cpu')
nodes, _, mass = graph_r(rosettes[0])
nodes = np.array(nodes)[:,1:]

for i in range(ncols):
    for j in range(nrows):
        axes[i, j].scatter(nodes[:,i*3+j], mass, s=0.5, color=cmap(0.8), label='data')
        axes[i, j].scatter(nodes[:,i*3+j], pred, s=0.5, color=cmap(0.2), label='GNN')
        axes[i, j].legend()
        axes[i, j].set_ylabel(r'$\log M_{*}$')
        axes[i, j].set_xlabel(f'{props[i*3+j]}')
        axes[i, j].grid()

plt.show()

In [ ]:
slope, intercept = np.polyfit(mass, pred.tolist(), 1)
r = slope*np.array(mass)+intercept

fig, ax = plt.subplots()
ax.scatter(mass, pred, s=0.8)
ax.scatter(mass, r, color='b', s=0.8)
ax.set_xlabel('Prediction')
ax.set_ylabel('Data')
plt.title(r'$\log M_{*}$ (GNN)')
plt.legend(['Data', 'Linear fit'])
plt.grid()
plt.show()